<a href="https://colab.research.google.com/github/AschHarwood/social_unrest/blob/main/social_unrest_predictions_v1_9_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#%pip install boto3

In [1]:
pip install tensorflow-addons

In [2]:
import tensorflow_addons as tfa

ModuleNotFoundError: ignored

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pandas as pd
import plotly.express as px
from copy import copy
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
import plotly.offline as py
import plotly.express as px
import plotly.graph_objects

#!pip install tensorflow-hub
#!pip install pandas-profiling[notebook]

import tensorflow_hub as hub

from pandas_profiling import ProfileReport


from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate

import pandas as pd
import numpy as np
import re
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn import preprocessing
from keras.losses import BinaryCrossentropy

tf.keras.backend.set_floatx('float64')

pd.set_option('display.max_columns', 500)


#unused preprocessing code

In [13]:
data_folder = '/content/drive/MyDrive/social_violence_predictions/data'

In [14]:
# response = client.list_objects(Bucket='politicalnews')
# file_list = []
# for obj in response['Contents']:
#     file_list.append(obj['Key'])


In [15]:
dataset_paths = file_list[350:349+45]

NameError: ignored

In [ ]:
# for dataset in dataset_paths:
#   filename = dataset[46:]
#   download_destination = data_folder + filename
#   client.download_file('politicalnews', dataset, download_destination)
#   print(dataset)

In [ ]:
# df = pd.read_feather('/content/drive/MyDrive/social_violence_predictions/data/Mobility_score_t-7.feather')
# df.set_index(['feature_date_range', 'target_date', 'state_fips', 'fips'], inplace=True)
# df = df.astype(float)
# df.reset_index(inplace=True)
# df.target_protest_count.value_counts()
# df.reset_index(drop=True, inplace=True)
# df.drop(df[df.fips=='nan'].index, axis=0, inplace=True)
# df.set_index(['feature_date_range', 'target_date', 'state_fips', 'fips'], inplace=True)

# Numeric function

# Getting Files

In [2]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [EarlyStopping(monitor='val_loss', patience=2)]
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


In [3]:
#get files
import os
uploadFileNames = []
for (sourceDir, dirname, filename) in os.walk('/content/drive/MyDrive/social_violence_predictions/data/numeric_time'):
    uploadFileNames.extend(filename)
    break

# Numeric preprocessing



In [4]:
def numeric_precessing(path):
  df = pd.read_feather(path)
  df.set_index(['feature_date_range', 'target_date', 'state_fips', 'fips'], inplace=True)
  df = df.astype(float)
  df.reset_index(inplace=True)
  df.target_protest_count.value_counts()
  df.reset_index(drop=True, inplace=True)
  df.drop(df[df.fips=='nan'].index, axis=0, inplace=True)
  df.set_index(['feature_date_range', 'target_date', 'state_fips', 'fips'], inplace=True)
  x = df.drop('target_protest_count', axis=1)
  y = df.target_protest_count
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, stratify=y)
  scaler = MinMaxScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)
  X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
  X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

  return X_train, y_train, X_test, y_test, df

# Compiling Models

In [5]:
def lstm(X_train, y_train, X_test, y_test):
  inputs = keras.layers.Input(shape = (X_train.shape[1], X_train.shape[2]))
  x = keras.layers.LSTM(100,return_sequences =  True)(inputs)
  x = keras.layers.Dropout(0.1)(x)
  x = keras.layers.LSTM(100, return_sequences = True)(x)
  x = keras.layers.Dropout(0.5)(x)
  x = keras.layers.LSTM(100, return_sequences = True)(x)
  x = keras.layers.LSTM(80, return_sequences = True)(x)
  x = keras.layers.Dropout(0.1)(x)
  x = keras.layers.LSTM(60, return_sequences = True)(x)
  x = keras.layers.LSTM(60, return_sequences = True)(x)
  x = keras.layers.Dropout(0.1)(x)
  x = keras.layers.LSTM(40, return_sequences = False)(x)
  x = tf.keras.layers.BatchNormalization()(x)
  outputs = keras.layers.Dense(1, activation = 'relu')(x)
  return inputs, outputs


In [10]:
from keras import backend as K

def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [6]:
model_inputs = []
model_outputs = []
X_train_list = []
X_test_list = []
y_train_list = []
y_test_list = []
df_list = []
for i, file in enumerate(uploadFileNames):
  path = '/content/drive/MyDrive/social_violence_predictions/data/numeric_time/'+file
  X_train, y_train, X_test, y_test, df = numeric_precessing(path)
  df_list.append(df)
  X_train_list.append(X_train)
  X_test_list.append(X_test)
  y_train_list.append(y_train)
  y_test_list.append(y_test)
  input, output = lstm(X_train, y_train, X_test, y_test)
  model_inputs.append(input)
  model_outputs.append(output)
  print(f'{path} completed')
  print(f'{i+1} of {len(uploadFileNames)+1} completed')
  


/content/drive/MyDrive/social_violence_predictions/data/numeric_time/Mobility_score_t-7.feather completed
1 of 39 completed
/content/drive/MyDrive/social_violence_predictions/data/numeric_time/cases_avg_per_100k_t-7.feather completed
2 of 39 completed
/content/drive/MyDrive/social_violence_predictions/data/numeric_time/cases_avg_t-7.feather completed
3 of 39 completed
/content/drive/MyDrive/social_violence_predictions/data/numeric_time/cases_t-7.feather completed
4 of 39 completed
/content/drive/MyDrive/social_violence_predictions/data/numeric_time/deaths_avg_per_100k_t-7.feather completed
5 of 39 completed
/content/drive/MyDrive/social_violence_predictions/data/numeric_time/deaths_avg_t-7.feather completed
6 of 39 completed
/content/drive/MyDrive/social_violence_predictions/data/numeric_time/deaths_t-7.feather completed
7 of 39 completed
/content/drive/MyDrive/social_violence_predictions/data/numeric_time/info_dyn_iri_RE_t-7.feather completed
8 of 39 completed
/content/drive/MyDrive/s

In [7]:
concat_layer = Concatenate()(model_outputs)
dense_1_cl = Dense(100, activation='relu')(concat_layer)
dense_1_cl = Dropout(0.5)(dense_1_cl)
dense_1_cl = Dense(80, activation='relu')(dense_1_cl)
dense_1_cl = Dropout(0.5)(dense_1_cl)
# dense_3_cl = Dense(40, activation='relu')(dense_2_cl)
# dense_4_cl = Dense(40, activation='relu')(dense_3_cl)
output = Dense(1, activation='relu')(dense_1_cl)

In [11]:
tf.keras.backend.clear_session()
validation_data = (X_test_list, y_test_list[0])
model = Model(inputs=model_inputs, outputs=output)

optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer,
             loss='mse',
             metrics=[coeff_determination, 'mse', 'mae', 'mape'])
print('model defined and compiled')

history = model.fit(X_train_list, y_train_list[0], epochs = 5,  callbacks=callbacks, validation_data = (X_test, y_test), shuffle = True )

model defined and compiled
Epoch 1/5
35660/35660 [==============================] - ETA: 0s - loss: 0.0040 - coeff_determination: 0.9016 - mse: 0.0040 - mae: 0.0033 - mape: 0.3407

ValueError: ignored

# Text Processing

In [ ]:
uploadFileNames = []
for (sourceDir, dirname, filename) in os.walk('/content/drive/MyDrive/social_violence_predictions/data/text'):
    uploadFileNames.extend(filename)
    break

In [ ]:
embedding = 'https://tfhub.dev/tensorflow/cord-19/swivel-128d/3'

In [ ]:
text = Input(shape=[], dtype=tf.string)
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)(text)
dense = Dense(16, activation='relu')(hub_layer)
dense = Dense(8, activation='relu')(dense)
dense = tf.keras.layers.BatchNormalization()(dense) 
output = Dense(4, activation='relu')(dense)

# Define LSTM Function

In [67]:
input, output = lstm(X_train, y_train, X_test, y_test)

In [68]:
model = keras.Model(inputs = inputs, outputs = outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
model.compile(optimizer=optimizer,
             loss='mse',
             metrics=[coeff_determination, 'mse', 'mae', 'mape'])

In [69]:
history = model.fit(X_train, y_train, epochs = 5,  callbacks=callbacks, validation_data = (X_test, y_test), shuffle = True )

Epoch 1/5
  517/35660 [..............................] - ETA: 6:26 - loss: 0.0038 - coeff_determination: 0.9101 - mse: 0.0038 - mae: 0.0031 - mape: 0.2780

KeyboardInterrupt: ignored

# LSTM

In [63]:
inputs = keras.layers.Input(shape = (X_train_reshaped.shape[1], X_train_reshaped.shape[2]))
x = keras.layers.LSTM(100,return_sequences =  True)(inputs)
#x = keras.layers.Dropout(0.1)(x)
x = keras.layers.LSTM(100, return_sequences = True)(x)
#x = keras.layers.Dropout(0.5)(x)
x = keras.layers.LSTM(100, return_sequences = True)(x)
x = keras.layers.LSTM(80, return_sequences = True)(x)
#x = keras.layers.Dropout(0.1)(x)
x = keras.layers.LSTM(60, return_sequences = True)(x)
x = keras.layers.LSTM(60, return_sequences = True)(x)
#x = keras.layers.Dropout(0.1)(x)
x = keras.layers.LSTM(40, return_sequences = False)(x)
x = tf.keras.layers.BatchNormalization()(x)
outputs = keras.layers.Dense(1, activation = 'relu')(x)

model = keras.Model(inputs = inputs, outputs = outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
model.compile(optimizer=optimizer,
             loss='mse',
             metrics=[coeff_determination, 'mse', 'mae', 'mape'])

In [65]:
history = model.fit(X_train, y_train, epochs = 150, batch_size = 100, callbacks=callbacks, validation_data = (X_test, y_test), shuffle = True )

Epoch 1/150
  418/11412 [>.............................] - ETA: 2:14 - loss: 0.0036 - coeff_determination: 0.7384 - mse: 0.0036 - mae: 0.0033 - mape: 0.9016

KeyboardInterrupt: ignored

# Dense Network

In [38]:
rsquare = tfa.metrics.r_square.RSquare()

In [41]:
num_input = Input(shape=(7,))
dense_layer_1_num = Dense(10, activation='relu')(num_input)
batch_out_num = tf.keras.layers.BatchNormalization()(dense_layer_1_num) 
num_output = Dense(10, activation='relu')(batch_out_num)
output = Dense(1, activation='sigmoid')(num_output)
model = Model(inputs=[num_input], outputs=output)

In [44]:
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer,
             loss='mean_squared_error',
             metrics=[coeff_determination, 'mse', 'mae', 'mape'])
print('model defined and compiled')


validation_data=(X_test, y_test)


print('training model')
history = model.fit(x=X_train, y=y_train, epochs=15, verbose=1, validation_split=0.2)
print('model training complete')

model defined and compiled
training model
Epoch 1/15
32604/32604 [==============================] - 131s 4ms/step - loss: 0.0048 - coeff_determination: -254166.8658 - mse: 0.0048 - mae: 0.0099 - mape: 6565875.5868 - val_loss: 0.0038 - val_coeff_determination: -2504.4062 - val_mse: 0.0038 - val_mae: 0.0052 - val_mape: 2032792.3592
Epoch 2/15
32604/32604 [==============================] - 122s 4ms/step - loss: 0.0040 - coeff_determination: -7772.4726 - mse: 0.0040 - mae: 0.0066 - mape: 3244832.6998 - val_loss: 0.0038 - val_coeff_determination: -4322.9962 - val_mse: 0.0038 - val_mae: 0.0059 - val_mape: 2789083.6724
Epoch 3/15
32604/32604 [==============================] - 129s 4ms/step - loss: 0.0040 - coeff_determination: -8499.0018 - mse: 0.0040 - mae: 0.0067 - mape: 3347268.0589 - val_loss: 0.0038 - val_coeff_determination: -3616.5885 - val_mse: 0.0038 - val_mae: 0.0055 - val_mape: 2375287.3666
Epoch 4/15
22650/32604 [===================>..........] - ETA: 33s - loss: 0.0040 - coeff_de

KeyboardInterrupt: ignored